### 선수 한 명 연봉

In [ ]:
import numpy as np
import pandas as pd
import random

import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from fake_useragent import UserAgent

import time
import csv
import os

In [ ]:
# fake_useragent 모듈을 통한 User-Agent 정보 생성
useragent = UserAgent()
print(useragent.chrome)

path = r'C:\Users\lg\Desktop\Scraping\chromedriver_83.exe'
driver = webdriver.Chrome(executable_path=path)

name = '양현종'
birth = '1988-03-01'
id = 77637

url = 'http://www.statiz.co.kr/player.php?opt=11&name={0}&birth={1}'.format(name, birth)
driver.get(url)

# #연봉 페이지로 이동
# driver.find_element_by_css_selector('body > div.wrapper > div.content-wrapper > div > section.content > div > div:nth-child(1) > div > div.col-xs-12.col-sm-8.col-md-8.col-lg-12 > div > div.table-responsive.no-padding.box > table > tbody > tr > td > a:nth-child(9) > i').click()

In [ ]:
table = driver.find_elements_by_css_selector('table')
lines=table[0].find_elements_by_xpath('/html/body/div[1]/div[1]/div/section[2]/div/div[2]/div/div/div/div[2]/table/tbody/tr')
print(len(lines))
salary = []
for l in range(0,len(lines)) :
    text = lines[l].text
    line = text.split(" ")
    salary.append(line[:2])
print(salary)
header = salary[0]
salary = pd.DataFrame(salary[1:], columns=header)

salary['연봉(만원)'].replace(salary['연봉(만원)'][salary['연봉(만원)'].str.contains(',') ==  False]
                            ,0, inplace=True) #연봉 정보 없는 경우 WAR 들어가는 것 0으로 변환
salary.rename(columns = {'연봉(만원)' : '{0}'.format(id)}, inplace = True)
salary['연도'] = salary['연도'].astype(int)

### 선수 전체 연봉

In [ ]:
import numpy as np
import pandas as pd
import random

import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from fake_useragent import UserAgent

import time
import csv
import os

In [ ]:
player_info = pd.read_csv('player_info.csv', encoding='euc-kr')
new_player_info = player_info.copy()

In [ ]:
new_player_info.info()
print(new_player_info.head())
print(new_player_info.tail())

In [ ]:
new_player_info['id'] = new_player_info['id'].astype(int)
new_player_info['birthdate'].replace(['년 ', '월 ', '일'],['-', '-', ''], regex=True, inplace=True)
new_player_info = new_player_info[['id', 'playername', 'birthdate']]

In [ ]:
new_player_info.info()
new_player_info.head()

In [ ]:
#생년월일 null인 행 제거
new_player_info['birthdate'].isna().sum(axis=0)
new_player_info.dropna(subset=['birthdate'], axis=0, inplace=True)
new_player_info.info()

In [ ]:
new_player_info.info()
print(new_player_info.head())
print(new_player_info.tail())

In [ ]:
total = pd.DataFrame(range(1997,2021), columns=['연도'])

# fake_useragent 모듈을 통한 User-Agent 정보 생성
useragent = UserAgent()
print(useragent.chrome)

path = r'C:\kkang\dev-tool\chromedriver.exe'
driver = webdriver.Chrome(executable_path=path)

for r in range(1000):
    for a,b,c in [new_player_info.iloc[r,:]] : 
        #print(a,b,c) #a:id b:이름 c:생년월일
        url = 'http://www.statiz.co.kr/player.php?opt=11&name={0}&birth={1}'.format(b,c)
        driver.get(url)
        time.sleep(2)       

        try :
            table = driver.find_elements_by_css_selector('table')
            lines=table[0].find_elements_by_xpath('/html/body/div[1]/div[1]/div/section[2]/div/div[2]/div/div/div/div[2]/table/tbody/tr')
            print(len(lines))


            salary = []
            for l in range(0,len(lines)) :
                text = lines[l].text
                line = text.split(" ")
                salary.append(line[:2])
            print(salary)

            header = salary[0]
            salary = pd.DataFrame(salary[1:], columns=header)
            salary['연봉(만원)'].replace(salary['연봉(만원)'][salary['연봉(만원)'].str.contains(',') ==  False]
                                        ,int(0), inplace=True) #연봉 정보 없는 경우 WAR 들어가는 것 0으로 변환
            salary.rename(columns = {'연봉(만원)' : '{0}'.format(a)}, inplace = True)
            salary['연도'] = salary['연도'].astype(int)

            total = total.merge(salary,how='left')
            print(total)

        except IndexError :
            print(b,'error')
            pass 
        r_number = random.randint(1,30)
        time.sleep(r_number)       

driver.quit()

total.info()
print(total.head())

### 연도별 성적 긁기 시도 중

### 단일시즌

In [ ]:
# fake_useragent 모듈을 통한 User-Agent 정보 생성
useragent = UserAgent()
print(useragent.chrome)

path = r'C:\kkang\dev-tool\chromedriver.exe'
driver = webdriver.Chrome(executable_path=path)

url = 'http://www.statiz.co.kr/stat.php?mid=stat&re=1&ys=2020&ye=2020&se=0&te=&tm=&ty=0&qu=all&po=0&as=&ae=&hi=&un=&pl=&da=1&o1=GP&o2=OutCount&de=1&lr=0&tr=&cv=&ml=1&sn=300&si=&cn='
driver.get(url)

table = driver.find_elements_by_xpath('/html/body/div[1]/div[1]/div/section[2]/div/div[2]/div[6]/div/div/div[2]')
print(table)
lines=table[0].find_elements_by_css_selector('tr')
print(len(lines))

#컬럼명 분리
col = list(lines[0].text.split("\n"))
col = ['순위', '이름', '연도팀', '출장'] +\
      col[1:-3]  +\
       ['WAR', 'WPA', 'ERA', 'FIP', 'WHIP', 'ERA+', 'FIP']
print(len(col))

#내용 긁기
stat = []
for l in range(2,len(lines)) :
    line = lines[l].text
    if '\n' in line :
        pass    
    else :
        line = lines[l].text.split(" ")
        if len(line) == 33 :
            stat.append(line)  
            r_number = random.randint(1,30)
            time.sleep(r_number) 
        else :
            line = [0]+line
            #print(line)
            stat.append(line)
            r_number = random.randint(1,30)
            time.sleep(r_number) 

print(len(stat))
driver.quit()

#df로 정리
stat = pd.DataFrame(stat, columns=col)
stat2 = stat.copy()
stat2 = stat2.drop(stat2.columns[0], axis=1, inplace=True)
stat2.info()

#저장
stat2.to_csv("C:/kkang/trying/stat_2020.csv", header=True, index=True, encoding='euc-kr')

### 연도별(2014-2020.08.11)

In [ ]:
# fake_useragent 모듈을 통한 User-Agent 정보 생성
useragent = UserAgent()
print(useragent.chrome)

path = r'C:\kkang\dev-tool\chromedriver.exe'
driver = webdriver.Chrome(executable_path=path)

years = list(range(2014,2015))
for y in years :
    url = 'http://www.statiz.co.kr/stat.php?opt=0&sopt=0&re=1&ys={0}&ye={0}&se=0&te=&tm=&ty=0&qu=all&po=0&as=&ae=&hi=&un=&pl=&da=1&o1=GP&o2=OutCount&de=1&lr=0&tr=&cv=&ml=1&sn=3000&si=&cn='.format(y)
    print(url)
    driver.get(url)

    table = driver.find_elements_by_xpath('/html/body/div[1]/div[1]/div/section[2]/div/div[2]/div[5]/div/div/div[2]')
    print(table)
    lines=table[0].find_elements_by_css_selector('tr')
    print(len(lines))
    
   ## 컬럼명 분리
    # col = list(lines[0].text.split("\n"))
    # cols = ['순위', '이름', '연도팀', '출장'] +\
    #     col[1:-3]  +\
    #     ['WAR', 'WPA', 'ERA', 'FIP', 'WHIP', 'ERA+', 'FIP']
    cols = ['순위','이름','연도팀','출장','출장','완투','완봉','선발',\
            '승','패','세','홀드','이닝','실점','자책','타자',\
            '안타','2타','3타','홈런','볼넷','고4','사구','삼진','보크','폭투',\
            'WAR','WPA','ERA','FIP','WHIP','ERA+','FIP']

    #내용 긁기
    stat = []
    for l in range(2,len(lines)-2) :
        line = lines[l].text
        if '\n' in line :
            pass    
        else :
            line = lines[l].text.split(" ")
            if len(line) == 33 :
                stat.append(line)  
                print(line)
            #     r_number = random.randint(1,3)
            #     time.sleep(r_number) 
            else :
                line = [0]+line
                print(line)
                stat.append(line)
                # r_number = random.randint(1,3)
                # time.sleep(r_number) 

    print(len(stat))

    #df로 정리
    stat = pd.DataFrame(stat, columns=cols)
    stat2 = stat.copy()
    stat2 = stat2.drop(stat2.columns[0], axis=1)

    #저장
    filename = "stat_{0}.csv".format(y)
    print(filename)
    stat2.to_csv(r"C:\kkang\trying\stat_by_year\{0}".format(filename), header=True, index=True, encoding='euc-kr')

    time.sleep(5)
driver.quit()    

### 2014년 이전 (2타, 3타, WPA X)

In [ ]:
# fake_useragent 모듈을 통한 User-Agent 정보 생성
useragent = UserAgent()
print(useragent.chrome)

# #크롬 headless 옵션
# options = webdriver.ChromeOptions()
# options.headless = True
# #options.add_argument('window-size=1920x1080')
# options.add_argument("disable-gpu")
# #headless 탐지 막기
# options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36")

path = r'C:\kkang\dev-tool\chromedriver.exe'
driver = webdriver.Chrome(executable_path=path)

years = list(range(20011,2014))
for y in years :
    url = 'http://www.statiz.co.kr/stat.php?opt=0&sopt=0&re=1&ys={0}&ye={0}&se=0&te=&tm=&ty=0&qu=all&po=0&as=&ae=&hi=&un=&pl=&da=1&o1=GP&o2=OutCount&de=1&lr=0&tr=&cv=&ml=1&sn=3000&si=&cn='.format(y)
    print(url)
    driver.get(url)

    table = driver.find_elements_by_xpath('/html/body/div[1]/div[1]/div/section[2]/div/div[2]/div[5]/div/div/div[2]')
    print(table)
    lines=table[0].find_elements_by_css_selector('tr')
    print(len(lines))
    
   # 컬럼명 분리
    # col = list(lines[0].text.split("\n"))
    # cols = ['순위', '이름', '연도팀', '출장'] +\
    #     col[1:-3]  +\
    #     ['WAR', 'WPA', 'ERA', 'FIP', 'WHIP', 'ERA+', 'FIP']
    # print(cols, len(cols))

    cols = ['순위','이름','연도팀','출장','출장','완투','완봉','선발',\
            '승','패','세','홀드','이닝','실점','자책','타자',\
            '안타','홈런','볼넷','고4','사구','삼진','보크','폭투',\
            'ERA','FIP','WHIP','ERA+','FIP', 'WAR']

    #내용 긁기
    stat = []
    for l in range(2,len(lines)-2) :
        line = lines[l].text
        if '\n' in line :
            pass    
        else :
            line = lines[l].text.split(" ")
            if len(line) == 30 :
                stat.append(line)  
                #print(line)
            #     r_number = random.randint(1,3)
            #     time.sleep(r_number) 
            else :
                line = [0]+line
                #print(line)
                stat.append(line)
                # r_number = random.randint(1,3)
                # time.sleep(r_number) 

    print(len(stat))

    #df로 정리
    stat = pd.DataFrame(stat, columns=cols)
    stat2 = stat.copy()
    stat2 = stat2.drop(stat2.columns[0], axis=1)

    #2014년 이후랑 열갯수 & 순서 맞추기 위해 새로운 df 생성
    stat2.insert(16,'2타', 0)
    stat2.insert(17,'3타', 0)
    stat2.insert(25,'WPA', 0)
    df1 = stat2.iloc[:, :25]
    df2 = pd.DataFrame(stat2['WAR'])
    df3 = stat2.iloc[:,25:-1]
    stat3 = pd.concat([df1, df2, df3], axis=1) 

    #저장
    filename = "stat_{0}.csv".format(y)
    print(filename)
    stat3.to_csv(r"C:\kkang\trying\stat_by_year\{0}".format(filename), header=True, index=True, encoding='euc-kr')

    time.sleep(5)
driver.quit()    